In [ ]:
import requests
import uuid
import datetime
import os

# Tentar importar o ID de sessão existente, se o arquivo ID.py existir
try:
    from ID import SESSION_ID # Tenta importar a variável SESSION_ID do arquivo ID.py
    print(f"ID de sessão '{SESSION_ID}' importado de ID.py") # Informa que o ID foi importado
except ImportError: # Se o arquivo ID.py não existir ou SESSION_ID não estiver definido
    SESSION_ID = None # Define SESSION_ID como None
    print("ID.py ou SESSION_ID não encontrado, um novo ID será gerado se necessário.") # Informa que um novo ID será gerado
except Exception as e: # Capturar outras possíveis exceções durante a importação
    SESSION_ID = None # Define SESSION_ID como None
    print(f"Erro ao importar ID.py: {e}. Um novo ID será gerado se necessário.") # Informa o erro na importação


def make_get_request(url):
  """Faz uma requisição GET para a URL e retorna a resposta ou um erro.""" # Docstring explicando a função
  try:
    response = requests.get(url) # Faz a requisição GET para a URL
    response.raise_for_status() # Levanta um erro para códigos de status ruins (4xx ou 5xx)
    return {'method': 'GET', 'status_code': response.status_code, 'headers': dict(response.headers)} # Retorna sucesso com detalhes
  except requests.exceptions.RequestException as e: # Captura erros de requisição
    return {'error': f"Erro na requisição GET para {url}: {e}"} # Retorna um dicionário de erro

def make_post_request(url, data=None):
  """Faz uma requisição POST para a URL com dados opcionais e retorna a resposta ou um erro.""" # Docstring explicando a função
  try:
    response = requests.post(url, data=data) # Faz a requisição POST para a URL com dados (opcional)
    response.raise_for_status() # Levanta um erro para códigos de status ruins (4xx ou 5xx)
    return {'method': 'POST', 'status_code': response.status_code, 'headers': dict(response.headers)} # Retorna sucesso com detalhes
  except requests.exceptions.RequestException as e: # Captura erros de requisição
    return {'error': f"Erro na requisição POST para {url}: {e}"} # Retorna um dicionário de erro

def make_head_request(url):
  """Faz uma requisição HEAD para a URL e retorna os headers ou um erro.""" # Docstring explicando a função
  try:
    response = requests.head(url) # Faz a requisição HEAD para a URL
    response.raise_for_status() # Levanta um erro para códigos de status ruins (4xx ou 5xx)
    return {'method': 'HEAD', 'status_code': response.status_code, 'headers': dict(response.headers)} # Retorna sucesso com detalhes
  except requests.exceptions.RequestException as e: # Captura erros de requisição
    return {'error': f"Erro na requisição HEAD para {url}: {e}"} # Retorna um dicionário de erro


def perform_all_requests(url, session_id=None): # Função principal para testar todos os métodos
  """
  Executa requisições GET, POST e HEAD para uma URL, coleta os resultados ou erros,
  e retorna um dicionário com os resultados, o método mais apropriado e um ID de sessão.
  Pode usar um session_id existente ou gerar um novo.
  Não imprime nada diretamente (exceto mensagens de importação/geração de ID).
  """ # Docstring explicando a função
  # Usar o session_id fornecido ou gerar um novo se nenhum for fornecido
  current_session_id = session_id if session_id is not None else str(uuid.uuid4()) # Usa o ID existente ou gera um novo UUID
  if session_id is None: # Se um novo ID foi gerado
      print(f"Gerando novo ID de sessão: {current_session_id}") # Informa que um novo ID foi gerado


  results = {} # Inicializa um dicionário para armazenar todos os resultados
  most_appropriate_method = "Nenhum método bem-sucedido" # Inicializa a variável para o método mais apropriado

  # Armazenar o ID da sessão nos resultados
  results['session_id'] = current_session_id # Adiciona o ID de sessão ao dicionário de resultados

  # Testando método GET
  get_info = make_get_request(url) # Chama a função para requisição GET
  results['GET'] = get_info # Armazena o resultado do GET
  if 'error' not in get_info and 200 <= get_info.get('status_code', 0) < 300: # Verifica se o GET foi bem-sucedido (sem erro e status 2xx)
      most_appropriate_method = 'GET' # Define GET como o método mais apropriado se for o primeiro sucesso

  # Testando método POST
  # Para um teste de POST mais realista, você pode precisar adicionar dados:
  # post_info = make_post_request(url, data={'chave': 'valor'}) # Exemplo de chamada POST com dados
  post_info = make_post_request(url) # Chama a função para requisição POST (sem dados neste exemplo)
  results['POST'] = post_info # Armazena o resultado do POST
  if 'error' not in post_info and 200 <= post_info.get('status_code', 0) < 300 and most_appropriate_method == "Nenhum método bem-sucedido": # Verifica se o POST foi bem-sucedido (sem erro e status 2xx) e se nenhum método anterior teve sucesso
      most_appropriate_method = 'POST' # Define POST como o método mais apropriado

  # Testando método HEAD
  head_info = make_head_request(url) # Chama a função para requisição HEAD
  results['HEAD'] = head_info # Armazena o resultado do HEAD
  if 'error' not in head_info and 200 <= head_info.get('status_code', 0) < 300 and most_appropriate_method == "Nenhum método bem-sucedido": # Verifica se o HEAD foi bem-sucedido (sem erro e status 2xx) e se nenhum método anterior teve sucesso
      most_appropriate_method = 'HEAD' # Define HEAD como o método mais apropriado

  results['most_appropriate_method'] = most_appropriate_method # Adiciona o método mais apropriado ao dicionário de resultados
  return results # Retorna o dicionário com todos os resultados e o resumo

# Função para logar os resultados em um arquivo
def log_request_results(results, url, filename="request_log.txt"):
    """
    Escreve os resultados das requisições em um arquivo de log.

    Args:
        results (dict): O dicionário retornado por perform_all_requests.
        url (str): A URL testada.
        filename (str): O nome do arquivo de log.
    """ # Docstring explicando a função
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Obtém o timestamp atual formatado
    with open(filename, "a") as f: # Abre o arquivo de log em modo de append ('a')
        f.write(f"--- Log gerado em: {timestamp} ---\n") # Escreve o timestamp do log
        f.write(f"--- Resultados para {url} ---\n") # Escreve a URL testada
        # Removido: f.write(f"ID de Sessão: {results.get('session_id', 'Não gerado')}\n") # Linha removida conforme solicitação anterior

        # Logando resultado do GET
        f.write(f"\n--- Método GET ---\n") # Título para a seção GET
        get_result = results.get('GET') # Obtém o resultado do GET do dicionário
        if get_result and 'error' in get_result: # Verifica se houve erro no GET
            f.write(f"{get_result['error']}\n") # Escreve a mensagem de erro
        elif get_result: # Se não houve erro e há resultados GET
            f.write(f"Status Code: {get_result['status_code']}\n") # Escreve o código de status
            f.write("Headers:\n") # Título para os headers
            for key, value in get_result.get('headers', {}).items(): # Itera sobre os headers
                f.write(f"  {key}: {value}\n") # Escreve cada header
        else: # Se não há resultado GET (não deveria acontecer com a lógica atual, mas é uma precaução)
            f.write("Resultado GET não disponível.\n") # Mensagem de erro interno

        # Logando resultado do POST
        f.write(f"\n--- Método POST ---\n") # Título para a seção POST
        post_result = results.get('POST') # Obtém o resultado do POST do dicionário
        if post_result and 'error' in post_result: # Verifica se houve erro no POST
            f.write(f"{post_result['error']}\n") # Escreve a mensagem de erro
        elif post_result: # Se não houve erro e há resultados POST
            f.write(f"Status Code: {post_result['status_code']}\n") # Escreve o código de status
            f.write("Headers:\n") # Título para os headers
            for key, value in post_result.get('headers', {}).items(): # Itera sobre os headers
                f.write(f"  {key}: {value}\n") # Escreve cada header
        else: # Se não há resultado POST
            f.write("Resultado POST não disponível.\n") # Mensagem de erro interno

        # Logando resultado do HEAD
        f.write(f"\n--- Método HEAD ---\n") # Título para a seção HEAD
        head_result = results.get('HEAD') # Obtém o resultado do HEAD do dicionário
        if head_result and 'error' in head_result: # Verifica se houve erro no HEAD
            f.write(f"{head_result['error']}\n") # Escreve a mensagem de erro
        elif head_result: # Se não houve erro e há resultados HEAD
            f.write(f"Status Code: {head_result['status_code']}\n") # Escreve o código de status
            f.write("Headers:\n") # Título para os headers
            for key, value in head_result.get('headers', {}).items(): # Itera sobre os headers
                f.write(f"  {key}: {value}\n") # Escreve cada header
        else: # Se não há resultado HEAD
            f.write("Resultado HEAD não disponível.\n") # Mensagem de erro interno

        f.write(f"\n--- Resumo ---\n") # Título para o resumo
        f.write(f"O método mais apropriado encontrado foi: {results.get('most_appropriate_method', 'Nenhum')}\n") # Escreve o método mais apropriado
        f.write("-" * 30 + "\n\n") # Separador para logs múltiplos no arquivo

# Exemplo de uso:
url = "https://www.anbima.com.br" # Define a URL a ser testada
# Passar o SESSION_ID importado para a função perform_all_requests
request_results = perform_all_requests(url, session_id=SESSION_ID) # Chama a função principal passando a URL e o ID de sessão (se importado)

# Extrair e salvar o ID de sessão em um arquivo Python separado (ID.py)
# Usar o ID que foi efetivamente utilizado na requisição (seja o importado ou o recém-gerado)
session_id_to_save = request_results.get('session_id', 'Não gerado') # Obtém o ID de sessão do resultado retornado
if session_id_to_save != 'Não gerado': # Verifica se um ID de sessão válido foi gerado/usado
    try:
        with open("ID.py", "w") as id_file: # Abre o arquivo ID.py em modo de escrita ('w')
            id_file.write(f"SESSION_ID = \"{session_id_to_save}\"\n") # Escreve a variável SESSION_ID no arquivo
        print(f"ID de sessão '{session_id_to_save}' salvo em 'ID.py'") # Informa que o ID foi salvo
    except IOError as e: # Captura erros de I/O ao escrever no arquivo
        print(f"Erro ao salvar ID de sessão em 'ID.py': {e}") # Informa o erro ao salvar

# Chamar a função de logging para salvar os resultados (sem o ID de sessão no log)
log_file = "google_request_log.txt" # Define o nome do arquivo de log
log_request_results(request_results, url, log_file) # Chama a função de logging com os resultados

print(f"Detalhes da requisição para {url} foram salvos em '{log_file}'") # Informa onde o log foi salvo

# Você pode opcionalmente imprimir o resumo no console também, se desejar
print(f"Resumo: O método mais apropriado encontrado foi: {request_results.get('most_appropriate_method', 'Nenhum')}") # Imprime o resumo no console

ID de sessão 'c0e7305d-0e8f-46dc-996a-93450e461522' importado de ID.py
ID de sessão 'c0e7305d-0e8f-46dc-996a-93450e461522' salvo em 'ID.py'
Detalhes da requisição para https://www.anbima.com.br foram salvos em 'google_request_log.txt'
Resumo: O método mais apropriado encontrado foi: GET
